# Quantum Devices becnhmarking

In [1]:
python_script = "../../z2chain/benchmark.py"
stdout_path = "../../z2chain/benchmark.log"
stderr_path = "../../z2chain/benchmark.err"
database_path = "../../z2chain/benchmarkdb_nnm.json"

## Launctl status

In [2]:
from utils.benchmarking import check_benchmark_status_mac

check_benchmark_status_mac(stdout_path)

The benchmark is NOT running


## Benchmark settings

In [3]:
from dateutil.relativedelta import relativedelta

J = 1
h = 0.05
lamb = 0.7
t_compute = 1.6
x_basis = False

L_arr = [52]
layers_arr = [5]
backends_arr = ["ibm_fez", "ibm_torino", "ibm_kyiv", "ibm_sherbrooke", "ibm_brussels"]

estimator_options = {
    "default_shots": 10000,
    "optimization_level": 0,
    "resilience_level": 0,
    "resilience": {
        "zne_mitigation": True,
        "measure_mitigation": True,
        "pec_mitigation": False,
        "zne": {
            "extrapolator": "linear", # zne_mitigation
        }
    },
    "dynamical_decoupling": {
        "enable": True,
        "sequence_type": "XY4"
    },
    "twirling": {
        "enable_gates": True,
        "enable_measure": True,
        "num_randomizations": "auto",
        "shots_per_randomization": "auto"
    }
}


seconds_to_start_interval = None
start_relativedeltas = [relativedelta(hour=0), relativedelta(hour=6), relativedelta(hour=12), relativedelta(hour=18)]

### Run benchmark

In [4]:
from utils.benchmarking import generate_and_execute_launchctl_file_mac
from cobos.tokens import ibm_token, group_instance

args = [
    database_path,
    ibm_token,
    "-i", group_instance,
    "-n", *L_arr,
    "-d", *layers_arr,
    "-b", *backends_arr,
    "-t", t_compute,
    "-j", J,
    "-g", h,
    "-l", lamb,
    "-o", estimator_options,
]

if x_basis: args.append("-x")

args = [str(arg) for arg in args]

generate_and_execute_launchctl_file_mac(
    python_script,
    *args,
    seconds_start_interval=seconds_to_start_interval,
    start_relativedeltas=start_relativedeltas,
    stdout_path=stdout_path,
    stderr_path=stderr_path,
)

### Stop benchmark

In [3]:
from utils.benchmarking import stop_benchmark_mac

stop_benchmark_mac()

## Database utilities

### Update database

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService
from cobos.tokens import ibm_token, group_instance
from utils.benchmarking import BenchmarkDB

benchmarkdb = BenchmarkDB(database_path)

service = QiskitRuntimeService(channel="ibm_quantum", token=ibm_token, instance=group_instance)
benchmarkdb.update_status(service, print_mode=True)

Updating benchmark index: 1/379, Job_dict: 5/5, Job: 1/1                                           

## Plot

### Date selection

In [1]:
from ipywidgets import DatetimePicker

start_time_dtpicker = DatetimePicker(description="Start time")
final_time_dtpicker = DatetimePicker(description="Final time")
display(start_time_dtpicker, final_time_dtpicker)

DatetimePicker(value=None, description='Start time')

DatetimePicker(value=None, description='Final time')